In [13]:
from scipy.interpolate import interp1d
import pandas as pd
import numpy as np

data_df = pd.read_excel(r"D:\Thesis\2024\OE-NMC10Ah-45SOC.xlsx")

data_df['Time_temp'] = data_df['Time (sec) '].astype(float)
data_df['Time_voltage'] = data_df['Time (second)'].astype(float)

# Calculate the overlapping time range
min_time = max(data_df['Time_temp'].min(), data_df['Time_voltage'].min())
max_time = min(data_df['Time_temp'].max(), data_df['Time_voltage'].max())

# reference time with a granularity of 0.1 seconds
reference_time = np.arange(min_time, max_time, 0.1)
reference_df = pd.DataFrame({'Reference Time (seconds)': reference_time})
#print(f"Length of reference_df: {len(reference_df)}")
#print(reference_df.head())

# Synchronize the data_df length with reference_df
if len(reference_df) < len(data_df):
    data_df = data_df.iloc[:len(reference_df)].copy()

data_df['Time(s)'] = reference_df['Reference Time (seconds)']

# Interpolate Temperature and Voltage
temp_interpolator = interp1d(
    data_df['Time_temp'], data_df['TC1 (°C)'], 
    kind='linear', bounds_error=False, fill_value='extrapolate'
)
reference_df['Interpolated Temperature'] = temp_interpolator(reference_df['Reference Time (seconds)'])
data_df['Temperature(°C)'] = reference_df['Interpolated Temperature']

voltage_interpolator = interp1d(
    data_df['Time_voltage'], data_df['Voltage (V)'], 
    kind='linear', bounds_error=False, fill_value='extrapolate'
)
reference_df['Interpolated Voltage'] = voltage_interpolator(reference_df['Reference Time (seconds)'])
data_df['Voltage(V)'] = reference_df['Interpolated Voltage']

# ensure consistent lengths
if len(data_df['Temperature(°C)'].dropna()) < len(data_df['Voltage(V)'].dropna()):
    data_df = data_df.iloc[:len(data_df['Temperature(°C)'].dropna())].copy()
else:
    data_df = data_df.iloc[:len(data_df['Voltage(V)'].dropna())].copy()

# Drop rows with NaN 
data_df.dropna(subset=['Temperature(°C)', 'Voltage(V)', 'Time(s)'], inplace=True)

required_columns = [
    'Cell No', 'Time(s)', 'Load (lb)', 'Voltage(V)', 'Temperature(°C)',
    'TC1 (°C)', 'TC2 (°C)', 'TC3 (°C)', 'TC4 (°C)', 
    'TC5 (°C)', 'TC6 (°C)', 'Observed Score', 
    'Observed Score ', 'Calculated Score', 'Capacity',
    'SOC', 'A', 'B', 'C', 'max T', 'rate T', 'voltage Difference'
]

for col in required_columns:
    if col not in data_df.columns:
        data_df[col] = None  # Add missing columns as empty

data_df = data_df[required_columns]

output_path = r"D:\Thesis\2024\OE-NMC10Ah-45SOC_ssss.xlsx"
data_df.to_excel(output_path, index=False)

print(f"Processed data saved to {output_path}")


Processed data saved to D:\Thesis\2024\OE-NMC10Ah-45SOC_ssss.xlsx
